In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/vinbigdata-chest-xray-abnormalities-detection/train'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import numpy as np
import warnings
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import patches

import cv2
import pandas as pd
import albumentations
from PIL import Image
from datetime import datetime
warnings.filterwarnings('ignore')

In [ ]:
dataset_dir = '../input/vinbigdata-chest-xray-abnormalities-detection'
output_dir = './'

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
        
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    return data

In [ ]:
def plot_img(img, title, cmap='gray'):
    plt.figure(figsize=(7,7))
    plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.show()

In [ ]:
def view_DICOM_metadata(img_id, test_data=False):
    if test_data:
        dicom = pydicom.read_file(f'{dataset_dir}/test/{img_id}.dicom')
    else:
        dicom = pydicom.read_file(f'{dataset_dir}/train/{img_id}.dicom')
    print(dicom)

In [ ]:
train_df = pd.read_csv(f'{dataset_dir}/train.csv')
train_df.head()

In [ ]:
finding_df = train_df[train_df['class_name'] != 'No finding']
finding_df.head()

In [ ]:
img_ids = finding_df['image_id'].unique()

In [ ]:
len(img_ids)

In [ ]:
view_DICOM_metadata(img_ids[0])

In [ ]:
shortlisted_img_ids = img_ids[:10]
og_imgs = [dicom2array(f'{dataset_dir}/train/{path}.dicom') for path in shortlisted_img_ids]

for img_as_arr, img_id in zip(og_imgs, shortlisted_img_ids):
    plot_img(img_as_arr, img_id)

In [ ]:
shortlisted_img_ids

In [ ]:
finding_df[finding_df["image_id"]=="9a5094b2563a1ef3ff50dc5c7ff71345"]

In [ ]:
def get_bb_info(df, img_id):
    bounding_boxes_info = df.loc[df["image_id"]==img_id, ['x_min', 'y_min', 'x_max', 'y_max', "class_id"]]

    bboxes = []
    for _, row in bounding_boxes_info.astype(np.int16).iterrows():
        bboxes.append(list(row))
    
    return bboxes

In [ ]:
label2color = { 0:("Aortic enlargement","#2a52be"),
                1:("Atelectasis","#ffa812"),
                2:("Calcification","#ff8243"),
                3:("Cardiomegaly","#4682b4"),
                4:("Consolidation","#ddadaf"),
                5:("ILD","#a3c1ad"),
                6:("Infiltration","#008000"),
                7:("Lung Opacity","#004953"),
                8:("Nodule/Mass","#e3a857"),
                9:("Other lesion","#dda0dd"),
               10:("Pleural effusion","#e6e8fa"),
               11:("Pleural thickening","#800020"),
               12:("Pneumothorax","#918151"),
               13:("Pulmonary fibrosis","#e75480")}

In [ ]:
def bounding_box_plotter(img_as_arr, img_id, bounding_boxes_info):
    fig = plt.figure(figsize=(7,7))
    ax = fig.add_axes([0,0,1,1])
    
    # plot the image
    plt.imshow(img_as_arr, cmap="gray")
    plt.title(img_id)

    # add the bounding boxes
    for row in bounding_boxes_info:
        # each row contains 'x_min', 'y_min', 'x_max', 'y_max', "class_id"
        xmin = row[0]
        xmax = row[2]
        ymin = row[1]
        ymax = row[3]

        width = xmax - xmin
        height = ymax - ymin
        
        edgecolor = label2color[row[4]][1]
        ax.annotate(label2color[row[4]][0], xy=(xmax - 40, ymin + 20))

        # add bounding boxes to the image
        rect = patches.Rectangle((xmin, ymin), width, height, edgecolor=edgecolor, facecolor='none')

        ax.add_patch(rect)

    plt.show()

In [ ]:
for img_as_arr, img_id in zip(og_imgs,shortlisted_img_ids):    
    bounding_boxes_info = get_bb_info(finding_df, img_id)
    bounding_box_plotter(img_as_arr, img_id, bounding_boxes_info)

In [ ]:
def generic_resize_image(df, image_id, image_path, transform_bb=True):
        
    # convert dicom to array
    img_arr = dicom2array(image_path)
    im_pil = Image.fromarray(img_arr)
    
    # training data
    if transform_bb:
        # create resize transform pipeline
        transform = albumentations.Compose([
            albumentations.Resize(height=512, width=512, always_apply=True)
        ], bbox_params=albumentations.BboxParams(format='pascal_voc')) 

        columns = ["x_min", "y_min", "x_max","y_max","class_id","class_name","rad_id"]
        bboxes = get_info(df, image_id, columns)

        transformed = transform(image=img_arr, bboxes=bboxes)
    else:
        transform = albumentations.Compose([
            albumentations.Resize(height=512, width=512, always_apply=True)
        ]) 

        transformed = transform(image=img_arr)

    # im_pil.size[0] = width, im_pil.size[1] = height
    return transformed, im_pil.size[0], im_pil.size[1]

In [ ]:
test_ids = []
for f in sorted(os.listdir(f"{dataset_dir}/test/")):
    test_ids.append(f)
    
transformed_test_dir = f"{output_dir}/transformed_data/test"
os.makedirs(transformed_test_dir, exist_ok=True)